<a href="https://colab.research.google.com/github/beyzassevval/Mental_Health-Social-Media-Dataset-/blob/main/Mental_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install catboost
!pip install xgboost
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              ExtraTreesRegressor, StackingRegressor)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')


In [17]:
# ----------------------------------------------------------------------
# 1. VERİ YÜKLEME
# ----------------------------------------------------------------------

FILE_NAME = "Mental_Health_and_Social_Media_Balance_Dataset.csv"

try:
    df = pd.read_csv(f"/content/{FILENAME}")
    print(f"✓ Veri yüklendi: {len(df)} satır")
except:
    df = pd.read_csv(FILE_NAME)
    print(f"✓ Veri yüklendi: {len(df)} satır")

# User_ID çıkar
df = df.drop('User_ID', axis=1, errors='ignore')


✓ Veri yüklendi: 500 satır


In [18]:
# ----------------------------------------------------------------------
# 2. GELİŞMİŞ ÖZELLİK MÜHENDİSLİĞİ
# ----------------------------------------------------------------------

print("\n🔧 Özellik mühendisliği yapılıyor...")

# Temel etkileşimler
df['Screen_Stress'] = df['Daily_Screen_Time(hrs)'] * df['Stress_Level(1-10)']
df['Sleep_Exercise'] = df['Sleep_Quality(1-10)'] * df['Exercise_Frequency(week)']
df['Screen_Sleep_Ratio'] = df['Daily_Screen_Time(hrs)'] / (df['Sleep_Quality(1-10)'] + 0.1)
df['Stress_Sleep_Ratio'] = df['Stress_Level(1-10)'] / (df['Sleep_Quality(1-10)'] + 0.1)

# İleri seviye etkileşimler
df['Health_Score'] = (
    df['Sleep_Quality(1-10)'] * 0.3 +
    df['Exercise_Frequency(week)'] * 0.3 -
    df['Stress_Level(1-10)'] * 0.2 -
    df['Daily_Screen_Time(hrs)'] * 0.1 +
    df['Days_Without_Social_Media'] * 0.1
)

df['Lifestyle_Balance'] = (
    (df['Exercise_Frequency(week)'] / (df['Daily_Screen_Time(hrs)'] + 1)) *
    (df['Sleep_Quality(1-10)'] / (df['Stress_Level(1-10)'] + 1))
)

df['Recovery_Index'] = (
    df['Days_Without_Social_Media'] * df['Sleep_Quality(1-10)'] /
    (df['Daily_Screen_Time(hrs)'] + 1)
)

# Polinom özellikleri
df['Screen_Squared'] = df['Daily_Screen_Time(hrs)'] ** 2
df['Stress_Squared'] = df['Stress_Level(1-10)'] ** 2
df['Sleep_Squared'] = df['Sleep_Quality(1-10)'] ** 2
df['Exercise_Squared'] = df['Exercise_Frequency(week)'] ** 2

# Logaritmik dönüşümler
df['Log_Screen'] = np.log1p(df['Daily_Screen_Time(hrs)'])
df['Log_Stress'] = np.log1p(df['Stress_Level(1-10)'])

# Yaş grupları
df['Age_Group_Young'] = (df['Age'] < 25).astype(int)
df['Age_Group_Middle'] = ((df['Age'] >= 25) & (df['Age'] < 40)).astype(int)
df['Age_Group_Senior'] = (df['Age'] >= 40).astype(int)

# Ekran süresi kategorileri
df['Low_Screen'] = (df['Daily_Screen_Time(hrs)'] < 3).astype(int)
df['Medium_Screen'] = ((df['Daily_Screen_Time(hrs)'] >= 3) &
                       (df['Daily_Screen_Time(hrs)'] < 6)).astype(int)
df['High_Screen'] = (df['Daily_Screen_Time(hrs)'] >= 6).astype(int)

# Stres kategorileri
df['Low_Stress'] = (df['Stress_Level(1-10)'] < 4).astype(int)
df['High_Stress'] = (df['Stress_Level(1-10)'] >= 7).astype(int)

# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['Gender', 'Social_Media_Platform'], drop_first=False)




🔧 Özellik mühendisliği yapılıyor...


In [19]:
# ----------------------------------------------------------------------
# 3. VERİ BÖLME VE ÖLÇEKLENDİRME
# ----------------------------------------------------------------------

X = df_encoded.drop('Happiness_Index(1-10)', axis=1)
y = df_encoded['Happiness_Index(1-10)']

# Daha büyük test seti
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Ölçeklendirme
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"✓ Train: {len(X_train)}, Test: {len(X_test)}")
print(f"✓ Toplam özellik sayısı: {X_train.shape[1]}")

✓ Train: 375, Test: 125
✓ Toplam özellik sayısı: 36


In [20]:

# ----------------------------------------------------------------------
# 4. ÇOK SAYIDA MODEL EĞİTİMİ
# ----------------------------------------------------------------------

print("\n🤖 Modeller eğitiliyor...\n")

models = {}

# XGBoost - Agresif parametreler
models['XGBoost'] = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=7,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.5,
    reg_lambda=1.0,
    random_state=42
)

# LightGBM - Hızlı ve etkili
models['LightGBM'] = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=8,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=0.3,
    random_state=42,
    verbose=-1
)

# CatBoost - Kategorik veriler için mükemmel
models['CatBoost'] = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.01,
    depth=8,
    l2_leaf_reg=5,
    random_seed=42,
    verbose=0
)

# Random Forest - Güçlü baseline
models['RandomForest'] = RandomForestRegressor(
    n_estimators=500,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

# Extra Trees - Daha fazla rastgelelik
models['ExtraTrees'] = ExtraTreesRegressor(
    n_estimators=500,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Gradient Boosting
models['GradientBoosting'] = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=6,
    min_samples_split=5,
    subsample=0.8,
    random_state=42
)

# Modelleri eğit ve değerlendir
results = {}
predictions = {}

for name, model in models.items():
    print(f"  → {name} eğitiliyor...")
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    predictions[name] = y_pred

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    results[name] = {'R2': r2, 'RMSE': rmse, 'MAE': mae}


🤖 Modeller eğitiliyor...

  → XGBoost eğitiliyor...
  → LightGBM eğitiliyor...
  → CatBoost eğitiliyor...
  → RandomForest eğitiliyor...
  → ExtraTrees eğitiliyor...
  → GradientBoosting eğitiliyor...


In [21]:
# ----------------------------------------------------------------------
# 5. STACKING ENSEMBLE (En Güçlü Yöntem)
# ----------------------------------------------------------------------

print("\n🔥 Stacking Ensemble oluşturuluyor...")

# Base modeller
base_models = [
    ('xgb', models['XGBoost']),
    ('lgbm', models['LightGBM']),
    ('cat', models['CatBoost']),
    ('rf', models['RandomForest']),
    ('et', models['ExtraTrees'])
]

# Meta model (Ridge Regression)
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(alpha=10.0),
    cv=5,
    n_jobs=-1
)

stacking_model.fit(X_train_scaled, y_train)
y_pred_stacking = stacking_model.predict(X_test_scaled)

r2_stacking = r2_score(y_test, y_pred_stacking)
rmse_stacking = np.sqrt(mean_squared_error(y_test, y_pred_stacking))
mae_stacking = mean_absolute_error(y_test, y_pred_stacking)

results['Stacking'] = {'R2': r2_stacking, 'RMSE': rmse_stacking, 'MAE': mae_stacking}
predictions['Stacking'] = y_pred_stacking


🔥 Stacking Ensemble oluşturuluyor...


In [22]:
# ----------------------------------------------------------------------
# 6. AĞIRLIKLI ENSEMBLE (Alternatif)
# ----------------------------------------------------------------------

# En iyi 5 modelin ağırlıklı ortalaması
weights = {
    'XGBoost': 0.25,
    'LightGBM': 0.25,
    'CatBoost': 0.20,
    'RandomForest': 0.15,
    'ExtraTrees': 0.15
}

y_pred_weighted = sum(predictions[name] * weight for name, weight in weights.items())

r2_weighted = r2_score(y_test, y_pred_weighted)
rmse_weighted = np.sqrt(mean_squared_error(y_test, y_pred_weighted))
mae_weighted = mean_absolute_error(y_test, y_pred_weighted)

results['Weighted_Ensemble'] = {'R2': r2_weighted, 'RMSE': rmse_weighted, 'MAE': mae_weighted}
predictions['Weighted_Ensemble'] = y_pred_weighted

In [23]:
# ----------------------------------------------------------------------
# 7. SONUÇLARI GÖSTER
# ----------------------------------------------------------------------

print("\n" + "="*70)
print("📊 MODEL PERFORMANS KARŞILAŞTIRMASI")
print("="*70)

results_df = pd.DataFrame(results).T.sort_values('R2', ascending=False)
print(results_df.to_string())

print("\n" + "="*70)
best_model_name = results_df.index[0]
best_r2 = results_df.iloc[0]['R2']
best_rmse = results_df.iloc[0]['RMSE']

print(f"🏆 EN İYİ MODEL: {best_model_name}")
print(f"   R² Skoru: {best_r2:.4f} ({best_r2*100:.2f}%)")
print(f"   RMSE: {best_rmse:.4f}")
print("="*70)

# Özellik önemleri (XGBoost için)
print("\n📈 En Önemli Özellikler (Top 15):")
feature_importance = pd.Series(
    models['XGBoost'].feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)
print(feature_importance.head(15))



📊 MODEL PERFORMANS KARŞILAŞTIRMASI
                         R2      RMSE       MAE
RandomForest       0.663335  0.909609  0.724516
Stacking           0.648672  0.929206  0.752093
CatBoost           0.626343  0.958279  0.782898
Weighted_Ensemble  0.602433  0.988464  0.791570
GradientBoosting   0.587854  1.006425  0.791510
ExtraTrees         0.584467  1.010551  0.773550
XGBoost            0.574983  1.022019  0.809664
LightGBM           0.524443  1.081078  0.862330

🏆 EN İYİ MODEL: RandomForest
   R² Skoru: 0.6633 (66.33%)
   RMSE: 0.9096

📈 En Önemli Özellikler (Top 15):
Stress_Sleep_Ratio                   0.320053
High_Screen                          0.101581
Screen_Stress                        0.082879
Stress_Squared                       0.054311
Screen_Sleep_Ratio                   0.051548
Stress_Level(1-10)                   0.038613
Social_Media_Platform_Facebook       0.025079
Sleep_Squared                        0.020666
Exercise_Squared                     0.017105
Daily_Scr

In [25]:
import pandas as pd
import numpy as np

def get_user_input():
    """Kullanıcıdan interaktif olarak veri alır"""

    print("\n" + "="*70)
    print("🎯 MUTLULUK ENDEKSİ TAHMİN SİSTEMİ")
    print("="*70)
    print("Lütfen aşağıdaki bilgileri girin:\n")

    # Yaş
    while True:
        try:
            age = int(input("📅 Yaşınız (18-80 arası): "))
            if 18 <= age <= 80:
                break
            print("   ⚠️  Lütfen 18-80 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Ekran Süresi
    while True:
        try:
            screen_time = float(input("📱 Günlük Ekran Süresi (saat, 0-24 arası): "))
            if 0 <= screen_time <= 24:
                break
            print("   ⚠️  Lütfen 0-24 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Uyku Kalitesi
    while True:
        try:
            sleep_quality = float(input("😴 Uyku Kalitesi (1-10 arası, 1=Çok Kötü, 10=Mükemmel): "))
            if 1 <= sleep_quality <= 10:
                break
            print("   ⚠️  Lütfen 1-10 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Stres Seviyesi
    while True:
        try:
            stress_level = float(input("😰 Stres Seviyesi (1-10 arası, 1=Çok Az, 10=Çok Fazla): "))
            if 1 <= stress_level <= 10:
                break
            print("   ⚠️  Lütfen 1-10 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Sosyal Medyasız Günler
    while True:
        try:
            days_without_sm = float(input("🚫 Son 30 günde sosyal medyadan uzak kaldığınız gün sayısı (0-30): "))
            if 0 <= days_without_sm <= 30:
                break
            print("   ⚠️  Lütfen 0-30 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Egzersiz Sıklığı
    while True:
        try:
            exercise_freq = float(input("🏃 Haftalık Egzersiz Sıklığı (gün, 0-7 arası): "))
            if 0 <= exercise_freq <= 7:
                break
            print("   ⚠️  Lütfen 0-7 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Cinsiyet
    print("\n👤 Cinsiyet Seçenekleri:")
    print("   1. Erkek (Male)")
    print("   2. Kadın (Female)")
    print("   3. Diğer (Other)")

    while True:
        try:
            gender_choice = int(input("Seçiminiz (1-3): "))
            if gender_choice == 1:
                gender = "Male"
                break
            elif gender_choice == 2:
                gender = "Female"
                break
            elif gender_choice == 3:
                gender = "Other"
                break
            else:
                print("   ⚠️  Lütfen 1, 2 veya 3 girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    # Sosyal Medya Platformu
    print("\n📲 En Çok Kullandığınız Sosyal Medya Platformu:")
    print("   1. Facebook")
    print("   2. Instagram")
    print("   3. YouTube")
    print("   4. TikTok")
    print("   5. LinkedIn")
    print("   6. X (Twitter)")

    platforms = {
        1: "Facebook",
        2: "Instagram",
        3: "YouTube",
        4: "TikTok",
        5: "LinkedIn",
        6: "X (Twitter)"
    }

    while True:
        try:
            platform_choice = int(input("Seçiminiz (1-6): "))
            if platform_choice in platforms:
                platform = platforms[platform_choice]
                break
            else:
                print("   ⚠️  Lütfen 1-6 arası bir değer girin.")
        except ValueError:
            print("   ⚠️  Lütfen geçerli bir sayı girin.")

    return {
        'age': age,
        'screen_time': screen_time,
        'sleep_quality': sleep_quality,
        'stress_level': stress_level,
        'days_without_sm': days_without_sm,
        'exercise_freq': exercise_freq,
        'gender': gender,
        'platform': platform
    }


def display_user_summary(user_data):
    """Kullanıcı verilerini özet olarak gösterir"""

    print("\n" + "="*70)
    print("📋 GİRDİĞİNİZ BİLGİLERİN ÖZETİ")
    print("="*70)
    print(f"  Yaş: {user_data['age']}")
    print(f"  Günlük Ekran Süresi: {user_data['screen_time']:.1f} saat")
    print(f"  Uyku Kalitesi: {user_data['sleep_quality']:.1f}/10")
    print(f"  Stres Seviyesi: {user_data['stress_level']:.1f}/10")
    print(f"  Sosyal Medyasız Günler: {user_data['days_without_sm']:.0f} gün")
    print(f"  Haftalık Egzersiz: {user_data['exercise_freq']:.1f} gün")
    print(f"  Cinsiyet: {user_data['gender']}")
    print(f"  Platform: {user_data['platform']}")
    print("="*70)


def analyze_lifestyle(user_data, prediction):
    """Kullanıcının yaşam tarzını analiz eder ve öneriler sunar"""

    print("\n" + "="*70)
    print("📊 YAŞAM TARZI ANALİZİ VE ÖNERİLER")
    print("="*70)

    # Ekran süresi analizi
    if user_data['screen_time'] > 6:
        print("\n⚠️  EKRAN SÜRESİ:")
        print("   Günlük ekran süreniz oldukça yüksek (6+ saat).")
        print("   💡 Öneri: Ekran süresini günde 4-5 saate düşürmeyi deneyin.")
    elif user_data['screen_time'] < 3:
        print("\n✅ EKRAN SÜRESİ:")
        print("   Ekran süreniz dengeli görünüyor!")
    else:
        print("\n👍 EKRAN SÜRESİ:")
        print("   Ekran süreniz makul seviyede.")

    # Uyku analizi
    if user_data['sleep_quality'] < 5:
        print("\n⚠️  UYKU KALİTESİ:")
        print("   Uyku kaliteniz düşük görünüyor.")
        print("   💡 Öneri: Düzenli uyku saatleri, karanlık oda, ekrandan uzak durun.")
    elif user_data['sleep_quality'] >= 7:
        print("\n✅ UYKU KALİTESİ:")
        print("   Uyku kaliteniz harika!")
    else:
        print("\n👍 UYKU KALİTESİ:")
        print("   Uyku kaliteniz orta seviyede, geliştirilebilir.")

    # Stres analizi
    if user_data['stress_level'] > 7:
        print("\n⚠️  STRES SEVİYESİ:")
        print("   Stres seviyeniz yüksek!")
        print("   💡 Öneri: Meditasyon, nefes egzersizleri, profesyonel destek.")
    elif user_data['stress_level'] < 4:
        print("\n✅ STRES SEVİYESİ:")
        print("   Stres seviyeniz düşük, harika!")
    else:
        print("\n👍 STRES SEVİYESİ:")
        print("   Stres seviyeniz normal aralıkta.")

    # Egzersiz analizi
    if user_data['exercise_freq'] < 2:
        print("\n⚠️  EGZERSİZ SIKLIĞI:")
        print("   Egzersiz sıklığınız düşük.")
        print("   💡 Öneri: Haftada en az 3-4 gün 30 dakika egzersiz yapın.")
    elif user_data['exercise_freq'] >= 4:
        print("\n✅ EGZERSİZ SIKLIĞI:")
        print("   Egzersiz sıklığınız mükemmel!")
    else:
        print("\n👍 EGZERSİZ SIKLIĞI:")
        print("   Egzersiz sıklığınız iyi, biraz artırabilirsiniz.")

    # Sosyal medya detoks
    if user_data['days_without_sm'] < 2:
        print("\n⚠️  SOSYAL MEDYA KULLANIMI:")
        print("   Sosyal medyadan neredeyse hiç uzaklaşmıyorsunuz.")
        print("   💡 Öneri: Haftada en az 1-2 gün dijital detoks yapın.")
    elif user_data['days_without_sm'] > 10:
        print("\n✅ SOSYAL MEDYA KULLANIMI:")
        print("   Sosyal medyadan düzenli uzaklaşıyorsunuz, harika!")

    # Genel değerlendirme
    print("\n" + "-"*70)
    print("🎯 GENEL DEĞERLENDİRME:")

    if prediction >= 8:
        print("   Mutluluk seviyeniz çok iyi! Mevcut alışkanlıklarınızı sürdürün.")
    elif prediction >= 6:
        print("   Mutluluk seviyeniz orta-iyi arası. Küçük iyileştirmelerle")
        print("   daha mutlu olabilirsiniz.")
    elif prediction >= 4:
        print("   Mutluluk seviyeniz ortalanın altında. Yaşam tarzınızda")
        print("   değişiklikler yapmanız faydalı olabilir.")
    else:
        print("   Mutluluk seviyeniz düşük görünüyor. Lütfen bir uzmandan")
        print("   destek almayı düşünün.")

    print("="*70)


def run_interactive_prediction():
    """Ana çalıştırma fonksiyonu"""

    # Model değişkenlerinin yüklenip yüklenmediğini kontrol et
    if 'stacking_model' not in globals() or 'scaler' not in globals():
        print("\n❌ HATA: Model henüz eğitilmedi!")
        print("Lütfen önce model eğitim kodunu çalıştırın.\n")
        return

    # Kullanıcıdan veri al
    user_data = get_user_input()

    # Özet göster
    display_user_summary(user_data)

    # Tahmin yap
    print("\n⏳ Tahmin yapılıyor...\n")

    prediction = predict_happiness_ultimate(
        age=user_data['age'],
        screen_time=user_data['screen_time'],
        sleep_quality=user_data['sleep_quality'],
        stress_level=user_data['stress_level'],
        days_without_sm=user_data['days_without_sm'],
        exercise_freq=user_data['exercise_freq'],
        gender=user_data['gender'],
        platform=user_data['platform']
    )

    # Yaşam tarzı analizi
    analyze_lifestyle(user_data, prediction)

    # Tekrar tahmin yapma seçeneği
    print("\n" + "="*70)
    repeat = input("\n🔄 Başka bir tahmin yapmak ister misiniz? (E/H): ")
    if repeat.lower() in ['e', 'evet', 'yes', 'y']:
        run_interactive_prediction()
    else:
        print("\n👋 Teşekkürler! Sağlıklı günler dileriz.\n")


# ----------------------------------------------------------------------
# PROGRAMI ÇALIŞTIR
# ----------------------------------------------------------------------

# Model eğitildiyse çalıştır
if 'stacking_model' in globals() and 'scaler' in globals():
    print("\n✅ Model hazır! İnteraktif tahmin başlatılıyor...\n")
    run_interactive_prediction()
else:
    print("\n" + "="*70)
    print("⚠️  BU KODU ÇALIŞTIRMADAN ÖNCE:")
    print("="*70)
    print("1. Önceki model eğitim kodunu çalıştırın")
    print("2. Model eğitimi tamamlandıktan sonra bu kodu çalıştırın")
    print("="*70)


✅ Model hazır! İnteraktif tahmin başlatılıyor...


🎯 MUTLULUK ENDEKSİ TAHMİN SİSTEMİ
Lütfen aşağıdaki bilgileri girin:

📅 Yaşınız (18-80 arası): 20
📱 Günlük Ekran Süresi (saat, 0-24 arası): 7
😴 Uyku Kalitesi (1-10 arası, 1=Çok Kötü, 10=Mükemmel): 9
😰 Stres Seviyesi (1-10 arası, 1=Çok Az, 10=Çok Fazla): 6
🚫 Son 30 günde sosyal medyadan uzak kaldığınız gün sayısı (0-30): 2
🏃 Haftalık Egzersiz Sıklığı (gün, 0-7 arası): 4

👤 Cinsiyet Seçenekleri:
   1. Erkek (Male)
   2. Kadın (Female)
   3. Diğer (Other)
Seçiminiz (1-3): 2

📲 En Çok Kullandığınız Sosyal Medya Platformu:
   1. Facebook
   2. Instagram
   3. YouTube
   4. TikTok
   5. LinkedIn
   6. X (Twitter)
Seçiminiz (1-6): 4

📋 GİRDİĞİNİZ BİLGİLERİN ÖZETİ
  Yaş: 20
  Günlük Ekran Süresi: 7.0 saat
  Uyku Kalitesi: 9.0/10
  Stres Seviyesi: 6.0/10
  Sosyal Medyasız Günler: 2 gün
  Haftalık Egzersiz: 4.0 gün
  Cinsiyet: Female
  Platform: TikTok

⏳ Tahmin yapılıyor...


🎯 MUTLULUK ENDEKSİ TAHMİNİ
  XGBoost:         9.13
  LightGBM:       